# 🛡️ Online Payments Fraud Detection - IBM Watson Deployment
### This notebook is for deploying the model to IBM Cloud

## Step 1 — Install IBM Watson ML Library

In [ ]:
!pip install ibm-watson-machine-learning
print('✅ IBM Watson ML installed!')

## Step 2 — Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pickle
import warnings
warnings.filterwarnings('ignore')
print('✅ Libraries imported!')

## Step 3 — Load and Prepare Data

In [ ]:
df = pd.read_csv('../data/fraud_dataset.csv')

le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

fraud_df     = df[df['isFraud'] == 1]
non_fraud_df = df[df['isFraud'] == 0].sample(n=30000, random_state=42)
balanced_df  = pd.concat([fraud_df, non_fraud_df]).sample(frac=1, random_state=42)

X = balanced_df.drop('isFraud', axis=1)
y = balanced_df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'✅ Data prepared! Training: {len(X_train):,} | Testing: {len(X_test):,}')

## Step 4 — Train and Save Model

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'✅ Accuracy: {acc*100:.2f}%')

with open('payments.pkl', 'wb') as f:
    pickle.dump(model, f)
print('✅ Model saved as payments.pkl')

## Step 5 — IBM Watson ML Credentials
*(Add your IBM Cloud credentials below)*

In [ ]:
# Add your IBM Cloud credentials here
wml_credentials = {
    "apikey"  : "YOUR_IBM_API_KEY",
    "url"     : "https://us-south.ml.cloud.ibm.com"
}

print('⚠️ Please replace YOUR_IBM_API_KEY with your actual IBM Cloud API key')
print('Get it from: https://cloud.ibm.com')

## Step 6 — Deploy to IBM Watson ML

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

# Set default space
client.set.default_space('YOUR_SPACE_ID')

# Define model metadata
model_props = {
    client.repository.ModelMetaNames.NAME: 'Online Payments Fraud Detection',
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:
        client.software_specifications.get_id_by_name('runtime-22.2-py3.10')
}

# Store model
stored_model = client.repository.store_model(
    model=model,
    meta_props=model_props,
    training_data=X_train,
    training_target=y_train
)

model_uid = client.repository.get_model_id(stored_model)
print(f'✅ Model stored! UID: {model_uid}')